In [1]:
import yaml
from google.cloud import bigquery


In [ ]:
config = yaml.load(open("credentials.yml"), yaml.Loader)
client = bigquery.Client.from_service_account_json(config["credentials_path"])


In [6]:
dataset_ref = bigquery.DatasetReference("bigquery-public-data", "stackoverflow")
posts_answers_ref = bigquery.TableReference(dataset_ref, "posts_answers")
posts_questions_ref = bigquery.TableReference(dataset_ref, "posts_questions")

posts_answers_table = client.get_table(posts_answers_ref)
posts_questions_table = client.get_table(posts_questions_ref)


In [7]:
client.list_rows(posts_answers_table, max_results=5).to_dataframe()


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,18,None,<p>For a table like this:</p>\n\n<pre><code>CR...,None,None,2,NaT,2008-08-01 05:12:44.193000+00:00,None,2016-06-02 05:56:26.060000+00:00,2016-06-02 05:56:26.060000+00:00,Jeff Atwood,126039,phpguy,<NA>,17,2,59,None,None
1,165,None,"<p>You can use a <a href=""http://sharpdevelop....",None,None,0,NaT,2008-08-01 18:04:25.023000+00:00,None,2019-04-06 14:03:51.080000+00:00,2019-04-06 14:03:51.080000+00:00,None,1721793,user2189331,<NA>,145,2,10,None,None
2,1028,None,<p>The VB code looks something like this:</p>\...,None,None,0,NaT,2008-08-04 04:58:40.300000+00:00,None,2013-02-07 13:22:14.680000+00:00,2013-02-07 13:22:14.680000+00:00,None,395659,user2189331,<NA>,947,2,8,None,None
3,1073,None,<p>My first choice would be a dedicated heap t...,None,None,0,NaT,2008-08-04 07:51:02.997000+00:00,None,2015-09-01 17:32:32.120000+00:00,2015-09-01 17:32:32.120000+00:00,None,45459,user2189331,<NA>,1069,2,29,None,None
4,1260,None,<p>I found the answer. all you have to do is a...,None,None,0,NaT,2008-08-04 14:06:02.863000+00:00,None,2016-12-20 08:38:48.867000+00:00,2016-12-20 08:38:48.867000+00:00,None,1221571,Jin,<NA>,1229,2,1,None,None


In [8]:
client.list_rows(posts_questions_table, max_results=5).to_dataframe()


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,320268,Html.ActionLink doesn’t render # properly,<p>When using Html.ActionLink passing a string...,<NA>,0,0,NaT,2008-11-26 10:42:37.477000+00:00,0,2009-02-06 20:13:54.370000+00:00,NaT,None,<NA>,Paulo,<NA>,None,1,0,asp.net-mvc,390
1,324003,Primitive recursion,<p>how will i define the function 'simplify' ...,<NA>,0,0,NaT,2008-11-27 15:12:37.497000+00:00,0,2012-09-25 19:54:40.597000+00:00,2012-09-25 19:54:40.597000+00:00,Marcin,1288,None,41000,None,1,0,haskell|lambda|functional-programming|lambda-c...,497
2,390605,While vs. Do While,<p>I've seen both the blocks of code in use se...,390608,0,0,NaT,2008-12-24 01:49:54.230000+00:00,2,2008-12-24 03:08:55.897000+00:00,NaT,None,<NA>,Unkwntech,115,None,1,0,language-agnostic|loops,11262
3,413246,Protect ASP.NET Source code,<p>Im currently doing some research in how to ...,<NA>,0,0,NaT,2009-01-05 14:23:51.040000+00:00,0,2009-03-24 21:30:22.370000+00:00,2009-01-05 14:42:28.257000+00:00,Tom Anderson,13502,Velnias,<NA>,None,1,0,asp.net|deployment|obfuscation,4823
4,454921,"Difference between ""int[] myArray"" and ""int my...",<blockquote>\n <p><strong>Possible Duplicate:...,454928,0,0,NaT,2009-01-18 10:22:52.177000+00:00,0,2009-01-18 10:30:50.930000+00:00,2017-05-23 11:49:26.567000+00:00,None,-1,Evan Fosmark,49701,None,1,0,java|arrays,798


In [28]:
query = """
    WITH q AS
    (
        SELECT id, title, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE EXTRACT(DATE FROM creation_date) = '2016-01-01' AND
              view_count > 1000
    )
    SELECT q.id, q.title, q.view_count, a.body, a.score
    FROM q
    LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
    ON q.id = a.parent_id
    ORDER BY q.title
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=30 * 10**10)  # 10 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results


,id,title,view_count,body,score
0,34559727,"""Cannot access NLS data files or invalid envir...",3965,<p>JMeter locale doesn't match Oracle locale. ...,0
1,34559727,"""Cannot access NLS data files or invalid envir...",3965,<p>I added the NLS_LANG variable in the enviro...,0
2,34559727,"""Cannot access NLS data files or invalid envir...",3965,"<p>In my case, I finally find that my Oracle v...",0
3,34560063,"""Operation not permitted"" running hello world ...",1331,<p>Figured it out.</p>\n\n<p>My code was on an...,1
4,34561093,"""Property access result unused - getters shoul...",1751,<p>This line:</p>\n\n<pre><code>self.motionMan...,9
...,...,...,...,...,...
1632,34553572,why is the button not vertically aligned?,1117,<p>Use following css. Make button position abs...,1
1633,34554092,why spring boot yaml have to be called applica...,4616,<p>Because by default spring boot looks for a ...,3
1634,34555467,withoutOverlapping() is not working in Laravel...,13608,<p>Just name your task with a call to <strong>...,13
1635,34555467,withoutOverlapping() is not working in Laravel...,13608,"<p>because these are different tasks, <code>wi...",4


In [36]:
# How long does it take for questions to receive answers?
query = """
    SELECT q.id AS q_id,
           MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
    FROM `bigquery-public-data.stackoverflow.posts_questions` as q
    LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` as a
    ON q.id = a.parent_id
    WHERE q.creation_date >= '2018-01-01' AND 
          q.creation_date < '2018-02-01' 
    GROUP BY q_id
    ORDER BY time_to_answer
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results

,q_id,time_to_answer
0,48509759,<NA>
1,48444452,<NA>
2,48126258,<NA>
3,48364283,<NA>
4,48065909,<NA>
...,...,...
161651,48325605,147696997
161652,48170466,147777409
161653,48236667,147877650
161654,48118092,148317285


In [37]:
# After signing up, how long does it take for users to first interact with the website?
query = """
    SELECT q.owner_user_id AS owner_user_id,
           MIN(q.creation_date) AS q_creation_date,
           MIN(a.creation_date) AS a_creation_date
    FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
    ON q.owner_user_id = a.owner_user_id
    WHERE q.creation_date >= '2019-01-01' AND
          q.creation_date < '2019-02-01' AND
          a.creation_date >= '2019-01-01' AND
          a.creation_date < '2019-02-01'
    GROUP BY owner_user_id
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results


,owner_user_id,q_creation_date,a_creation_date
0,872344,2019-01-24 18:59:09.870000+00:00,2019-01-01 04:06:02.283000+00:00
1,3231395,2019-01-01 07:30:17.550000+00:00,2019-01-01 07:49:26.797000+00:00
2,10832127,2019-01-22 19:31:52.437000+00:00,2019-01-01 06:08:41.347000+00:00
3,7585847,2019-01-01 02:51:24.567000+00:00,2019-01-01 10:59:28.247000+00:00
4,4181340,2019-01-01 11:04:40.870000+00:00,2019-01-01 11:06:57.580000+00:00
...,...,...,...
21518,10103070,2019-01-08 05:51:51.410000+00:00,2019-01-13 04:43:00.087000+00:00
21519,3680098,2019-01-22 07:21:23.103000+00:00,2019-01-24 08:15:24.037000+00:00
21520,4208407,2019-01-27 18:16:16.140000+00:00,2019-01-23 21:34:40.890000+00:00
21521,8359201,2019-01-31 06:28:31.750000+00:00,2019-01-31 12:19:26.060000+00:00


In [39]:
dataset_ref = bigquery.DatasetReference("bigquery-public-data", "stackoverflow")
posts_answers_ref = bigquery.TableReference(dataset_ref, "posts_answers")
posts_questions_ref = bigquery.TableReference(dataset_ref, "posts_questions")
users_ref = bigquery.TableReference(dataset_ref, "users")

posts_answers_table = client.get_table(posts_answers_ref)
posts_questions_table = client.get_table(posts_questions_ref)
users_table = client.get_table(users_ref)


In [41]:
users_table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('display_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('about_me', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('age', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('last_access_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('location', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('reputation', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('up_votes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('down_votes', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('views', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('profile_image_url', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('website_url', 'STRING', 'NULLABLE', None, None, (), None)]

In [46]:
# Users who joined the site in January 2019. You want to track their activity on the site.
# When did they post their first questions and answers, if ever?
query = """
    SELECT u.id AS id, 
           MIN(q.creation_date) AS q_creation_date, 
           MIN(a.creation_date) AS a_creation_date
    FROM `bigquery-public-data.stackoverflow.users` AS u
    FULL JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
    ON u.id = q.owner_user_id
    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
    on u.id = a.owner_user_id
    WHERE u.creation_date >= '2019-01-01' AND
          u.creation_date < '2019-02-01'
    GROUP BY id
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results


,id,q_creation_date,a_creation_date
0,10956896,2019-01-23 17:23:46.273000+00:00,NaT
1,10948004,NaT,NaT
2,10933621,NaT,NaT
3,10875497,2019-01-06 16:24:29.580000+00:00,NaT
4,10910552,NaT,NaT
...,...,...,...
141701,10917737,NaT,NaT
141702,10928755,NaT,NaT
141703,10969582,NaT,NaT
141704,10933655,NaT,NaT


In [50]:
# How many distinct users posted on January 1, 2019?
query = """
    SELECT q.owner_user_id
    FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
    WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
    UNION DISTINCT
    SELECT a.owner_user_id
    FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
    WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
"""

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)  # 10 GB
query_job = client.query(query, job_config=safe_config)
query_results = query_job.to_dataframe()
query_results


,owner_user_id
0,6767417
1,579856
2,9947723
3,9694487
4,10632037
...,...
4347,259889
4348,292166
4349,10850918
4350,27876
